In [1]:
import datasets
from datasets import load_dataset

In [2]:
# from huggingface_hub import notebook_login
# notebook_login()

In [3]:
magnetics_dataset=load_dataset("nlp-magnets/magnetics_corpus_all",\
                               split='train',cache_dir="/l/users/huseyin.ucar/.cache/huggingface/datasets")

In [4]:
len(magnetics_dataset)


173360

In [5]:
#remove None in abstract.
magnetics_dataset=magnetics_dataset.filter(lambda x:x['abstract'] is not None)

In [6]:
len(magnetics_dataset), magnetics_dataset[10],magnetics_dataset

(150684,
 {'title': 'Structural, morphological and magnetic analysis of Cd–Co–S dilute magnetic semiconductor nanofilms',
  'abstract': 'Cd1−x Co x S dilute magnetic semiconductor nanofilms (0≤x≤0.08at%) deposited by chemical bath deposition have been investigated using grazing angle x-ray diffraction, atomic force microscopy and vibrating sample magnetometer. The introduction of Co2+ ions in CdS structure induces structural disorders and hence, results in degradation of crystallinity. The crystallite size, interplanar spacing and lattice parameter ratio decrease with increasing Co2+ concentration in CdS. The diamagnetic state of CdS disappears with increase in Co concentration and films with x>0.02 exhibit ferromagnetism. This may be explained in terms of the spin–orbit interactions and Co2+ ion induced the lattice defects and phase separation.',
  'text': '1 Introduction Dilute magnetic semiconductors (DMS) have received a lot of theoretical and experimental attention because of thei

In [7]:
def compute_text_length(example):
    return {"text_length":len(example["text"].split()),
    "abstract_length":len(example["abstract"].split())}

In [8]:
magnetics_dataset=magnetics_dataset.map(compute_text_length)

In [9]:
magnetics_dataset=magnetics_dataset.filter(lambda x:x["abstract_length"]>50)

In [10]:
len(magnetics_dataset)

144581

In [11]:
def text_filter(example):
    return (example["text"]!="Text is not string." and example["text"]!="Read document failed.")

In [12]:
ds1=magnetics_dataset.filter(text_filter)
ds2=magnetics_dataset.filter(lambda x:x["text"]=="Text is not string.")

In [13]:
def replace_text_not_string(example):
    return {"text":""}

In [14]:
ds2

Dataset({
    features: ['title', 'abstract', 'text', 'doi', 'text_length', 'abstract_length'],
    num_rows: 13185
})

In [15]:
ds2=ds2.map(replace_text_not_string)

In [16]:
magnetics_dataset=datasets.concatenate_datasets([ds1,ds2])

In [17]:
len(magnetics_dataset)

144581

In [18]:
def combine_title_abs_text(example):
    return {"title_abs_text":example["title"]+" "+example["abstract"]+" "+example["text"]}

In [19]:
magnetics_dataset=magnetics_dataset.map(combine_title_abs_text)

In [20]:
len(magnetics_dataset)

144581

In [21]:
magnetics_dataset=magnetics_dataset.shuffle(seed=0).train_test_split(train_size=0.9, seed=0)

In [22]:
from transformers import AutoTokenizer
model_checkpoint="m3rg-iitd/matscibert"
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

(…)ibert/resolve/main/tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

(…)iitd/matscibert/resolve/main/config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

(…)g-iitd/matscibert/resolve/main/vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

(…)d/matscibert/resolve/main/tokenizer.json:   0%|          | 0.00/467k [00:00<?, ?B/s]

(…)ert/resolve/main/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [23]:
tokenized_sample=tokenizer("This is simple text.")

In [24]:
tokenizer.decode(tokenized_sample['input_ids'])

'[CLS] this is simple text. [SEP]'

In [25]:
[tokenized_sample.word_ids()[i] for i in range(len(tokenized_sample['input_ids']))]

[None, 0, 1, 2, 3, 4, None]

In [26]:
def tokenize(example):
    result=tokenizer(example['title_abs_text'])
    if tokenizer.is_fast:
        result['word_ids']=[result.word_ids(i) for i in range(len(result['input_ids']))]
    return result

In [27]:
tokenized_dataset=magnetics_dataset.map(tokenize,batched=True,\
    remove_columns=["title","abstract","text","doi","text_length","abstract_length","title_abs_text"],num_proc=8)

Map (num_proc=8):   0%|          | 0/130122 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/14459 [00:00<?, ? examples/s]

In [28]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 130122
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 14459
    })
})

In [29]:
tokenizer.model_max_length

1000000000000000019884624838656

In [30]:
chunk_size=512

In [31]:
tokenized_samples = tokenized_dataset['train'][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Text {idx} length: {len(sample)}'")

'>>> Text 0 length: 7624'
'>>> Text 1 length: 94'
'>>> Text 2 length: 3604'


In [32]:
concatenated_examples={
    k:sum(tokenized_samples[k],[]) for k in tokenized_samples.keys()
}
total_length=len(concatenated_examples['input_ids'])
print(f"'>>> Concatenated texts length: {total_length}'")

'>>> Concatenated texts length: 11322'


In [33]:
chunks={
    k:[t[i:i+chunk_size] for i in range(0,total_length,chunk_size)]\
        for k,t in concatenated_examples.items()
}


for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 58'


In [34]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [35]:
chunked_magnetics_ds=tokenized_dataset.map(group_texts,batched=True,num_proc=8)
chunked_magnetics_ds

Map (num_proc=8):   0%|          | 0/130122 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/14459 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 972026
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 106523
    })
})

In [36]:
chunked_magnetics_ds.save_to_disk("/l/users/huseyin.ucar/NLP_magnets_data/chunked_ds_matscibert")

Saving the dataset (0/23 shards):   0%|          | 0/972026 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/106523 [00:00<?, ? examples/s]

In [37]:
!du -h /l/users/huseyin.ucar/NLP_magnets_data/chunked_ds_matscibert

3.1G	/l/users/huseyin.ucar/NLP_magnets_data/chunked_ds_matscibert/train
62M	/l/users/huseyin.ucar/NLP_magnets_data/chunked_ds_matscibert/test
3.1G	/l/users/huseyin.ucar/NLP_magnets_data/chunked_ds_matscibert
